In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stroke"
cohort = "GSE48424"

# Input paths
in_trait_dir = "../../input/GEO/Stroke"
in_cohort_dir = "../../input/GEO/Stroke/GSE48424"

# Output paths
out_data_file = "../../output/preprocess/Stroke/GSE48424.csv"
out_gene_data_file = "../../output/preprocess/Stroke/gene_data/GSE48424.csv"
out_clinical_data_file = "../../output/preprocess/Stroke/clinical_data/GSE48424.csv"
json_path = "../../output/preprocess/Stroke/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Severe preeclampsia: new blood circulating biomarkers"
!Series_summary	"Background: Preeclampsia (PE) is a placental disease characterized by hypertension and proteinuria in pregnant women, which is associated with a high maternal and infantile morbidity. However, circulating biomarkers able to predict the prognosis of PE are lacking."
!Series_summary	"Methods: Thirty-eight women were included in the study. They consisted of 19 patients with PE (13 with severe PE and 6 women with non-severe PE) and 19 gestational age-matched normal pregnancy controls. We measured coagulation pathway, endothelial responses and microparticle release and circulating gene expression in PE patient groups and normotensive controls."
!Series_summary	"Results: The measurement of markers associated with coagulation pathway, endothelial activation and circulating microparticles enabled to discriminate PE from normal pregnancy but were not sufficient to distinguish severe fr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import re
import json
from typing import Optional, Callable, Dict, Any, List

# Analyze the data availability and setup conversion functions

# 1. Determine gene expression data availability
# Based on the background information about preeclampsia research with gene expression mentioned,
# the dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Stroke), we need to determine if it's available in this dataset
# Looking at the sample characteristics dictionary, key 1 has 'disease status' which shows preeclampsia vs healthy
# This dataset is about preeclampsia, not stroke, so it's not directly relevant to our trait
trait_row = None

# For age, the sample characteristics dictionary key 3 has multiple age values
age_row = 3

# For gender, there's no explicit mention in the sample characteristics
# Based on the background information, this is a study of pregnant women, so all subjects are female
# Since gender is constant, we consider it not available for our analysis
gender_row = None

# 2.2 Data Type Conversion Functions
# Since trait data is not available for our stroke study, we'll still define a function
# but it won't be used in further analysis
def convert_trait(value):
    # This won't be used since trait_row is None, but we define it as required
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value).lower()
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    if 'preeclampsia' in value_str or 'pe' in value_str:
        return 1  # affected
    elif 'healthy' in value_str or 'normal' in value_str:
        return 0  # unaffected
    else:
        return None

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value)
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    try:
        return float(value_str)  # Age is continuous
    except ValueError:
        return None

def convert_gender(value):
    # This won't be used since gender_row is None, but we define it as required
    # All subjects in this study are female
    return 0  # Female = 0

# 3. Save Metadata
# Determine if trait data is available (it's not for our stroke study)
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this substep since trait_row is None (clinical data for stroke is not available)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Stroke/GSE48424/GSE48424_series_matrix.txt.gz


Gene data shape: (41000, 36)
First 20 gene/probe identifiers:
Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, they appear to be Agilent microarray probe IDs (A_23_P format)
# These are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apopt

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the annotation dataframe contain probe IDs and gene symbols
# Based on the preview, 'ID' contains probe identifiers (like A_23_P100001) that match the gene expression data
# 'GENE_SYMBOL' contains the human gene symbols we need to map to

# 2. Get a gene mapping dataframe with the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping dataframe:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
# This function distributes probe values to genes and aggregates across multiple probes
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("First few rows of gene expression dataframe:")
print(gene_data.head())

# Optional: Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (30936, 2)
First few rows of gene mapping dataframe:
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN
Gene expression dataframe shape after mapping: (18488, 36)
First few rows of gene expression dataframe:
          GSM1177776  GSM1177777  GSM1177778  GSM1177779  GSM1177780  \
Gene                                                                   
A1BG       13.191333   11.225844   13.194065   12.276784   11.186062   
A1BG-AS1    7.266653    6.900023    6.468707    6.847638    7.573267   
A1CF        2.140806    2.034748    4.770635    4.082254    1.471878   
A2LD1       6.948963    6.910073    6.219797    7.122556    7.056264   
A2M         3.595800    3.276375    3.455032    3.937070    3.930950   

          GSM1177781  GSM1177782  GSM1177783  GSM1177784  GSM1177785  ...  \
Gene                                                                  ...   
A1BG

Gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE48424.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
# Use the gene_data variable that was already loaded in Step 4
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. No clinical data available (trait_row is None from previous steps)
# We can't proceed with linking clinical and genetic data
print("No trait data available. Can't proceed with linking clinical and genetic data.")

# Create a minimal dataframe for validation purposes 
# (since we need a valid DataFrame when is_final=True)
empty_df = pd.DataFrame({"dummy": [0]})

# Since trait data is not available, the dataset is not usable for our purposes
# We pass is_biased=True to indicate unusable data
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,
    is_biased=True,  # Setting is_biased to True since missing trait data makes dataset unusable
    df=empty_df,
    note="No stroke-specific trait data available in this dataset. Contains gene expression data from peripheral blood of older adults, but without clear stroke indicators."
)
print("Dataset deemed not usable due to missing trait data.")

Original gene data shape: (18488, 36)
Gene data shape after normalization: (18247, 36)


Normalized gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE48424.csv
No trait data available. Can't proceed with linking clinical and genetic data.
Abnormality detected in the cohort: GSE48424. Preprocessing failed.
Dataset deemed not usable due to missing trait data.
